<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/ChrillePille/Lab_3/ChrillePille_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Christian Weigelt**

# **Abstract**

This lab consisted of the implementation of a selection of iterative methods for solving linear equations, with varying properties.
Test code was written to verify function outputs with random, test cases.
In the introduction section, the functions are given a brief description, both of input/output, and what is to be tested.
In the method section, short definition of the functions are given, and their respective implementation and test function is presented.
In the results section, the output of the test cases is presented.

# **Set up environment**

In [208]:
import numpy as np
import math

# **Introduction**

In this lab, the assignment was to implement 3 functions, with input and output as defined in the lab instructions, as well as write code tests to test output.
  
1. Function: Jacobi iteration for Ax=b

  Input: matrix A, vector b</br>
  Output: vector x

  Test: convergence of residual || Ax-b ||, || x-y || for manufactured/exact solution y 
</br>
2. Function: Gauss-Seidel iteration for Ax=b

  Input: matrix A, vector b</br>
  Output: vector x

  Test: convergence of residual || Ax-b ||, || x-y || for manufactured/exact solution y
</br>
3. Function: Newton's method for scalar nonlinear equation f(x)=0

  Input: scalar function f(x)</br>
  Output: real number x

  Test: convergence of residual |f(x)|, |x-y| for manufactured/exact solution y
</br>

# **Method**

Here the code for the assignment is provided.

###Jacobi iteration for $Ax = b$
Function 1 is 'Jacobi iteration for $Ax = b$'

From chapter 7.7, we get the Jacobi iteration matrix as $M_J = -D^{-1}(A - D) = (I - D^{-1}A)$,
and the formula for iteratively calculating $x^{(k+1)}$ is:

 $x^{(k+1)} = (I - D^{-1}A)x^{k} + D^{-1}b $

 Where $c = D^{-1}b$ is constant, and we start with $x^{(0)} = c$.

 For testing we will use a preconditioner for convergence, with $B$ being an approximate inverse of $A$

In [141]:
def jacobi_iteration(a, b, tol):
  # Assume a is square (n*n) and b is a vector of length n
  i = np.identity(a.shape[0])
  d = np.diag(np.diag(a))
  # since d is a diagonal matrix, the inverse is simple each element e replaced by 1/e
  d_inv = np.linalg.inv(d) 
  c = np.dot(d_inv, b)
  x = c
  while np.linalg.norm(np.dot(a, x) - b) > tol:
    x = np.dot((i - np.matmul(d_inv, a)), x) + c
  return x

To test the above code, we can run the following test function:

In [142]:
def test_jacobi_iteration():
  print("Testing jacobi_iteration()")
  a = np.random.rand(10, 10)
  y = np.random.rand(10)
  b = np.dot(a, y)

  # Use preconditioner for convergence
  pre = np.linalg.inv(a)
  pre = pre * 0.95
  a = np.matmul(pre, a)
  b = np.dot(pre, b)

  x = jacobi_iteration(a, b, 1e-10)

  #test if |Ax - b| is close to 0
  assert np.isclose(np.linalg.norm(np.dot(a, x) - b), 0.) == True, "incorrect result from jacobi_iteration"

  #test if |x - y| is close to 0
  assert np.isclose(np.linalg.norm(x - y), 0.) == True, "solution not close to manufactured solution"

###Gauss-Seidel iteration for Ax=b

Function 2 is 'Gauss-Seidel iteration for Ax=b'

From chapter 7.7, we get the Gauss-Seidel iteration matrix as $M_GS = -L^{-1}(A - L) = (I - L^{-1}A)$,
and the formula for iteratively calculating $x^{(k+1)}$ is:

 $x^{(k+1)} = (I - L^{-1}A)x^{k} + L^{-1}b $

 Where $c = L^{-1}b$ is constant, and we start with $x^{(0)} = c$.

 For testing we will use a preconditioner for convergence, with $B$ being an approximate inverse of $A$

In [162]:
def gauss_seidel_iteration(a, b, tol):
  # Assume a is square (n*n) and b is a vector of length n
  i = np.identity(a.shape[0])
  l = np.zeros_like(a)
  for i in range(a.shape[0]):
    for j in range(a.shape[0]):
      if i >= j:
        l[i, j] = a[i, j]
  #since l is a lower triangular matrix, it is easily invertible by forward substitution
  l_inv = np.linalg.inv(l)
  c = np.dot(l_inv, b)
  x = c
  while np.linalg.norm(np.dot(a, x) - b) > tol:
    x = np.dot((i - np.matmul(l_inv, a)), x) + c
  return x

To test the above code, we can run the following test function:

In [165]:
def test_gauss_seidel_iteration():
  print("Testing gauss_seidel_iteration()")
  a = np.random.rand(10, 10)
  y = np.random.rand(10)
  b = np.dot(a, y)

  # Use preconditioner for convergence
  pre = np.linalg.inv(a)
  pre = pre * 0.95
  a = np.matmul(pre, a)
  b = np.dot(pre, b)

  x = gauss_seidel_iteration(a, b, 1e-9)

  #test if |Ax - b| is close to 0
  assert np.isclose(np.linalg.norm(np.dot(a, x) - b), 0.) == True, "incorrect result from gauss_seidel_iteration"
  
  #test if |x - y| is close to 0
  assert np.isclose(np.linalg.norm(x - y), 0.) == True, "solution not close to manufactured solution"

###Newton's method for scalar nonlinear equation f(x)=0

Function 3 is 'Newton's method for scalar nonlinear equation f(x)=0'

The function finds a solution to the nonlinear equation $f(x) = 0$ by derivating the function and using a tangent line to the function to find a solution x that solves the equation.

A polynomial function is used, by use of 'numpy.polynomial'.


In [183]:
def newton_nonlinear(f, tol):
  # Guess x0 = 0
  x = 0
  while np.abs(f(x)) > tol:
    df = np.polyder(f)
    x -= f(x)/df(x)
  return x

To test the above code, we can run the following test function:

In [205]:
def test_newton_nonlinear():
  print("Testing newton_nonlinear()")
  coeffs = np.random.randint(30, size=4)
  f = np.poly1d(coeffs, True)
  x = newton_nonlinear(f, 1e-10)

  # Test if f(x) is very close to zero, i.e. root found
  assert np.isclose(f(x), 0), "No root f(x)=0 found"

  # Check if x is close to a given root to the polynomial function
  match = False
  for c in coeffs:
    if np.isclose(c, x):
      match = True
  assert match == True, "x not one of f's given roots?"

###Testing
Then to perform all the tests, we can run the following code:

In [206]:
def run_all_tests():
  test_jacobi_iteration()
  test_gauss_seidel_iteration()
  test_newton_nonlinear()
  print("All tests OK")

if __name__ == '__main__':
  run_all_tests()

Testing jacobi_iteration()
Testing gauss_seidel_iteration()
Testing newton_nonlinear()
All tests OK


# **Results**

Running the test cases here in google colab, after importing required libraries, defining all functions, etc., generates the following output:
```
Testing jacobi_iteration()
Testing gauss_seidel_iteration()
Testing newton_nonlinear()
All tests OK
```
From which we can see that all test cases were passed.

# **Discussion**

In this lab, I think that the hardest part, for me at least, was to understand how functions were supposed to be represented in python. I made an attempt to solve the extra assigment of implementing newton's method for vector nonlinear equations, but figuring out an appropriate way to represent and handle vector functions generally was not something I had time nor energy to figure out by myself.